In [1]:
import psycopg2

try:
    # Conexão com o banco de dados
    connection = psycopg2.connect(
        host="26.84.71.27",       
        database="DataScienceProj",
        user="postgres",     
        password="SteamDB",    
        port="5432",
        options="-c client_encoding=UTF8"  # Configuração da codificação
    )

    # Criação de um cursor para executar comandos SQL
    cursor = connection.cursor()

    # Exemplo de execução de uma consulta
    cursor.execute("SELECT version();")
    version = cursor.fetchone()
    print("Versão do PostgreSQL:", version)

except Exception as error:
    # Tratamento de erro com codificação segura
    print("Erro ao conectar ao banco de dados:", str(error).encode('utf-8', errors='replace').decode())


Versão do PostgreSQL: ('PostgreSQL 17.2 on x86_64-windows, compiled by msvc-19.41.34123, 64-bit',)


In [5]:
import requests
import psycopg2  # ou o conector que você estiver utilizando

# Conexão com o banco de dados
connection =  connection = psycopg2.connect(
        host="26.84.71.27",       
        database="DataScienceProj",
        user="postgres",     
        password="SteamDB",    
        port="5432",
        options="-c client_encoding=UTF8" 
    )
cursor = connection.cursor()

# Requisição para obter dados dos países
response = requests.get('https://restcountries.com/v3.1/all?fields=name,area')
data = response.json()

# Inserir dados na tabela
for country in data:
    country_name = country['name']['common']
    area = country['area']
    
    # Definindo a query com a cláusula ON CONFLICT DO NOTHING, baseada no ID
    sqlCountry = """
    INSERT INTO PUBLIC.COUNTRY (COUNTRY_NAME, ID)
    VALUES (%s, %s)
    ON CONFLICT (ID) DO NOTHING;
    """
    val = (country_name, area)  # 'area' será o valor para o campo ID
    
    # Executando a query
    cursor.execute(sqlCountry, val)

# Commit e fechamento da conexão
connection.commit()
cursor.close()
connection.close()



In [33]:
import requests
from bs4 import BeautifulSoup
import psycopg2  

# Conexão com o banco de dados
connection = psycopg2.connect(
    host="26.84.71.27",       
    database="DataScienceProj",
    user="postgres",     
    password="SteamDB",    
    port="5432",
    options="-c client_encoding=UTF8" 
)
cursor = connection.cursor()

# URL do site da Wikipédia
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_Human_Development_Index'

# Fazendo a requisição para o site
response = requests.get(url)

# Verificando se a requisição foi bem-sucedida
if response.status_code == 200:
    # Parseando o conteúdo HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Encontrando todos os elementos <a> dentro da tabela
    links = soup.select('table.wikitable a[href^="/wiki/"]')  # Seleciona links que começam com "/wiki/"

    # Lista para armazenar os nomes dos países
    nomes_paises = []

    for link in links:
        nome = link.get_text(strip=True)  # Obtém o texto do link
        if nome:  # Verifica se o texto não está vazio
            nomes_paises.append(nome)

    # Limitando a 10 resultados
    nomes_paises_limitados = nomes_paises[:10]

    # Usando um loop for com range para iterar sobre os nomes limitados
    for i in range(1, len(nomes_paises_limitados) + 1):
        nome = nomes_paises_limitados[i - 1]  # Acessando o elemento na posição correta
        print(f"{i}: {nome}") 
        # Vinculando o país com o banco de dados
        cursor.execute("SELECT id FROM public.country WHERE upper(country_name) = upper(%s)", (nome,))
        resultado = cursor.fetchone()

        if resultado:
            pais_id = resultado[0]
            # Inserindo o registro
            cursor.execute("INSERT INTO public.Best_country_live (id, country_id) VALUES (%s,%s)", (i,pais_id,))
        else:
            print(f'País {nome} não encontrado no banco de dados.')

    connection.commit()
else:
    print('Erro ao acessar o site:', response.status_code)

cursor.close()
connection.close()


1: Switzerland
2: Norway
3: Iceland
4: Hong Kong
5: Denmark
6: Sweden
7: Ireland
8: Germany
9: Singapore
10: Netherlands


In [32]:
import requests
from bs4 import BeautifulSoup
import psycopg2  

# Conexão com o banco de dados
connection = psycopg2.connect(
    host="26.84.71.27",       
    database="DataScienceProj",
    user="postgres",     
    password="SteamDB",    
    port="5432",
    options="-c client_encoding=UTF8" 
)
cursor = connection.cursor()

# URL do site da Wikipédia
url = 'https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population'

# Fazendo a requisição para o site
response = requests.get(url)

# Verificando se a requisição foi bem-sucedida
if response.status_code == 200:
    # Parseando o conteúdo HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Encontrando todos os elementos <a> dentro da tabela
    links = soup.select('table.wikitable a[href^="/wiki/"]')  # Seleciona links que começam com "/wiki/"

    # Lista para armazenar os nomes dos países
    nomes_paises = []

    for link in links:
        nome = link.get_text(strip=True)  # Obtém o texto do link
        if nome:  # Verifica se o texto não está vazio
            nomes_paises.append(nome)

    # Limitando a 10 resultados
    nomes_paises_limitados = nomes_paises[:10]

    # Usando um loop for com range para iterar sobre os nomes limitados
    for i in range(1, len(nomes_paises_limitados) + 1):
        nome = nomes_paises_limitados[i]  # Acessando o elemento na posição correta
        print(f"{i}: {nome}") 
        # Vinculando o país com o banco de dados
        cursor.execute("SELECT id FROM public.country WHERE upper(country_name) = upper(%s)", (nome,))
        resultado = cursor.fetchone()

        if resultado:
            pais_id = resultado[0]
            # Inserindo o registro
            cursor.execute("INSERT INTO public.Populous_countries (id, country_id) VALUES (%s,%s) on conflict (id) do nothing", (i,pais_id,))
        else:
            print(f'País {nome} não encontrado no banco de dados.')

    connection.commit()
else:
    print('Erro ao acessar o site:', response.status_code)

cursor.close()
connection.close()


1: China
2: India
3: United States
4: Indonesia
5: Pakistan
6: Nigeria
7: Brazil
8: Bangladesh
9: Russia


IndexError: list index out of range